In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def reset_graph(seed = 318):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
# Question1:reset_graph(seed=318)函数的两个作用？
# 清除默认图的堆栈，并设置全局图为默认图

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [21]:
mnist = input_data.read_data_sets('../CNN/data/mnist', one_hot=True)
# Question2:
# 2.1、train, validation, test的大小；

# 2.2、train集合里images、labels的shape

# Question3：
# 3.1 one_hot=False时，train集合里labels的shape

Extracting ../CNN/data/mnist\train-images-idx3-ubyte.gz
Extracting ../CNN/data/mnist\train-labels-idx1-ubyte.gz
Extracting ../CNN/data/mnist\t10k-images-idx3-ubyte.gz
Extracting ../CNN/data/mnist\t10k-labels-idx1-ubyte.gz


In [26]:
# print(mnist.train.epochs_completed)
# print(mnist.train.images.shape)
# print(mnist.train.labels.shape)
print(mnist.train.next_batch(10))
# print(mnist.train.num_examples)
# print(mnist.test.images.shape)
# print(mnist.validation.images.shape)
# print(mnist.train.labels.shape)

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]))


### constants

In [22]:
# image
img_height = 28
img_width = 28
img_channels = 1
img_classes = 10

# neural network
n_steps = img_height
n_inputs = img_width
n_neurons = 20
n_outputs = img_classes

# other hyperparameters
learning_rate = 1e-4

### placeholer variables

In [35]:
reset_graph()
# Question 4: reset_graph()的位置能否再向下移？
# 不能，否则建立的图又会被删除

In [36]:
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

# Question 5: 当one_hot == False时，placeholder variable y该如何定义？
# y = tf.placeholder(tf.float32,[None,])

### models

In [37]:
# RNN
basic_cell = tf.contrib.rnn.BasicRNNCell(n_neurons, activation = tf.nn.tanh)
outputs, states = tf.nn.dynamic_rnn(basic_cell, x, dtype = tf.float32)

# Output Layer
logits = tf.layers.dense(states, n_outputs, activation = tf.nn.softmax)

# Question 6：
# 6.1 什么是python的序列解包？
# 1、一般情况 2、元祖序列解包 3、列表序列解包 4、字典序列解包
# 6.2 描述一下该网络的拓扑结构。
# 总线性拓扑结构
# 6.3 tf.contrib module是各什么性质的模块？
# 高阶API

### loss

In [38]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = logits)
loss = tf.reduce_mean(cross_entropy)

# Question 7：
# 7.1 当Output layer的activation=None时，cross_entropy该使用哪个函数来替代tf.nn.softmax_cross_entropy_with_logits()？
# 最小二倍差？
# 7.2 在tf.nn.softmax_cross_entropy_with_logits()函数里，为何使用命名参数？
# 区分预测值和参考值？
# hints：1）交叉熵损失确切定义；2）交叉熵为什么这样定义？3）避免传错参数。

### optimizer & train_op

In [39]:
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

### evaluation models

In [40]:
correct = tf.equal(tf.argmax(y, 1), tf.argmax(logits, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# Question 8：
# 8.1 解释计算得到correct向量的过程。
# 8.2 为何要用tf.cast()函数？
# 把布尔值转变成数字
# 8.3 one_hot = False时，给出正确得到correct tensor的代码。
# correct = tf.nn.in_top_k(prediction,y,1)

### Build a Session

In [41]:
n_epochs = 1001
batch_size = 20

# Quesiton8：
# 8.1 batch_size大小对模型占用的显存或内存是否有影响？
# 有
# 8.2 n_pochs取1000不好么？
# 含头不含尾

init = tf.global_variables_initializer()

# 统计量
loss_summary = tf.summary.scalar('loss', loss)
acc_summary = tf.summary.scalar('acc', accuracy)
merged = tf.summary.merge_all()

# 保存统计信息的磁盘文件
logdir = '../log_dir'
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# build check points
checkpoints_path = '../mnist_models'
saver = tf.train.Saver()

In [33]:
with tf.Session() as sess:
    
    sess.run(init)
#     saver.restore(sess, checkpoints_path)
# Question 9:
#     9.1 何时注释掉sess.run(init)？
#     接着上次的checkpoints运行时
#     9.2 何时注释掉saver.restore(sess, checkpoints_path)？
#     最开始运行时
    
    for epoch in range(n_epochs):
        # for training
        x_batch, y_batch = mnist.train.next_batch(batch_size)
        
        x_batch_rnn = np.reshape(x_batch, [-1, n_steps, n_inputs])
#         Question 10:
#             10.1 为何需要np.reshape()?顺便解释该程序用rnn来训练字符识别时，输入数据时怎样流入计算图的？
#              使得输入维度匹配模型的输入，每一行每一行的输入，n_steps等于图片高度
#             10.2 解释将np.reshape()更换成tf.reshape()出错的原因？
#             tf.reshape()的结果是一个tensor，不能作为输入
            
        train_acc, _ = sess.run([accuracy, train], feed_dict = {x: x_batch_rnn, y: y_batch})
        
        # for testing
        x_batch, y_batch = mnist.test.next_batch(batch_size)
        x_batch_rnn = np.reshape(x_batch, [-1, n_steps, n_inputs])
        [merged_str, test_acc] = sess.run([merged, accuracy],
                           feed_dict = {x:x_batch_rnn, y: y_batch})
        file_writer.add_summary(merged_str, epoch)
        
        if epoch % 100 == 0:
            print(f'epoch = {epoch}; train_acc = {train_acc}; test_acc = {test_acc}')
        
    saver.save(sess, checkpoints_path)
# Question 11:
# 11.1将上一行代码取消缩进，直接顶格，会出现错误，解释错误原因？
# 必须在会话运行的时候才能保存图
# 11.2将上一行代码增加缩进量，放到for epoch in range(n_epochs)循环，为什么不好？
# 每次循环都保存checkpoints，增加内存

file_writer.close()

# Question 12：
# 12.1 给出Jupyter notebook将代码分成小的cell的优点？
# 边写代码边运行
# 12.2 给出Jupyter notebook将代码分成小的cell的缺点？
# 要一个一个运行，比较麻烦
# 12.3 如何避免在jupyter notebook里，错误已经修改正确，代码依然会出现bug？
# 重新运行已经修改过的代码
# 12.4 在本例代码中，从“models”下面的cell重新执行代码会出错，而从“placeholer variables”下面的cell开始重新执行代码则无错。
# 结合出错信息，对照在MLP、CNN程序中是否也会又类似错误。寻找现象差异的原因？
# 本代码中错误提示：outputs,states那一行代码

epoch = 0; train_acc = 0.0; test_acc = 0.05000000074505806
epoch = 100; train_acc = 0.05000000074505806; test_acc = 0.05000000074505806
epoch = 200; train_acc = 0.10000000149011612; test_acc = 0.05000000074505806
epoch = 300; train_acc = 0.10000000149011612; test_acc = 0.10000000149011612
epoch = 400; train_acc = 0.15000000596046448; test_acc = 0.15000000596046448
epoch = 500; train_acc = 0.25; test_acc = 0.30000001192092896
epoch = 600; train_acc = 0.20000000298023224; test_acc = 0.25
epoch = 700; train_acc = 0.10000000149011612; test_acc = 0.25
epoch = 800; train_acc = 0.20000000298023224; test_acc = 0.15000000596046448
epoch = 900; train_acc = 0.25; test_acc = 0.20000000298023224
epoch = 1000; train_acc = 0.05000000074505806; test_acc = 0.15000000596046448
